In [36]:
import pandas as pd
from math import log
import numpy as np
from scipy.stats import beta
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import logit
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('framingham_heart_disease.csv')

# Part A

# Hypothesis question:
## Third Experimental question - Does BMI (X) change between gender (Y)

## Question 1

In [3]:
df_observed = df.dropna().sample(n=100, random_state=1)
df_old_observation = df.drop(df_observed.index).dropna().sample(n=1000, random_state=1)

## Question 2 a

Let's define $\tau$ as the median of the bmi, and j=1 as male and j=2 as female

In [4]:
tau = df_observed["BMI"].median()
print(f'tau={tau}')

tau=24.915


As we seen in class, the MLE estimator of $\psi$ is $log(\frac{(n - {X^{2}_{\tau}}){X^{1}_{\tau}}}{(m - {X^{1}_{\tau}}){X^{2}_{\tau}}})$ where:<br> m is number of males,<br> n is number of females,<br> $X^{1}_{\tau}=\sum_{i=1}^{m}I\{x_{i}^1>\tau\}$,<br> $X^{2}_{\tau}=\sum_{i=1}^{n}I\{x_{i}^{2}>\tau\}$.

In [54]:
def calcualte_phi_mle(temp_df, tau):
    x_1 = len(temp_df[(temp_df["male"] == 1) & (temp_df["BMI"] > tau)])
    x_2 = len(temp_df[(temp_df["male"] == 0) & (temp_df["BMI"] > tau)])
    m = len(temp_df[temp_df["male"] == 1])
    n = len(temp_df[temp_df["male"] == 0])
    phi = log((x_1*(n - x_2))/(x_2)*(m- x_1))
    return phi, n, m

In [55]:
phi, n, m = calcualte_phi_mle(df_observed, tau)

In [56]:
print(f"Number of male: {n}, number of female: {m}. MLE estimation of Phi: {np.around(phi,2)}")

Number of male: 53, number of female: 47. MLE estimation of Phi: 6.56


We can see that the log odd ratio is bigger than zero, from that we can infer that the males BMI are more likely to be bigger than $\tau$ compared to females BMI.

Let's find the confident interval of $\psi$ using (quantile) Bootsrap

In [8]:
B = 500
alpha = 0.05
quantile_list = []
for _ in range(B):
    temp_df = df_observed.sample(n=len(df_observed), replace=True)
    phi, _, _ = calcualte_phi_mle(temp_df, tau)
    quantile_list.append(phi)
a = np.quantile(quantile_list, q=(alpha/2))
b = np.quantile(quantile_list, q=(1-(alpha/2)))
print(f"The confident interval is: [{np.around(a, 2)}, {np.around(b,2)}]")

The confident interval is: [5.83, 7.27]


And indeed, the MLE estimator that we just calculated is inside the interval. Moreover, we can see that the confident interval is short and informative, it suggest that the BMI of male is more likely to be above $\tau$ than BMI of females.

## Question 2 b

We will assume that: <br> $\pi(p_1)=1I\{0\le{p_1}\le1\}$, <br> $\pi(p_2)=1I\{0\le{p_2}\le1\}$ <br> ${\mathcal{L}}_m(p_1)={p_1}^{X^{1}_{\tau}}{(1-p_1)}^{m-X^{1}_{\tau}}$ <br> ${\mathcal{L}}_n(p_2)={p_2}^{X^{2}_{\tau}}{(1-p_2)}^{n-X^{2}_{\tau}}$ <br> and as we seen in lecture the postriors are: <br> $p1|X^1\sim{Beta(X^{1}_{\tau} + 1, m - X^{1}_{\tau} +1)}$ <br> $p2|X^2\sim{Beta(X^{2}_{\tau} + 1, n - X^{2}_{\tau} +1)}$<br> We will compute the estimator using simulation by sampling $p_1$ and $p_2$ from their aposterior distributions, and average the simulated phi.

In [9]:
def simulate_p_beta(x_1, x_2, m, n, jeffery=False):
    a1 = x_1 + 0.5 if jeffery else x_1 + 1
    b1 = m - x_1 + 0.5 if jeffery else m - x_1 + 1
    a2 = x_2 + 0.5 if jeffery else x_2 + 1 
    b2 = n - x_2 + 0.5 if jeffery else  n - x_2 + 1
    return beta.rvs(a1, b1), beta.rvs(a2, b2)

In [57]:
B = 10000
phi_list = []
x_1 = len(df_observed[(df_observed["male"] == 1) & (df_observed["BMI"] > tau)]["BMI"])
x_2 = len(df_observed[(df_observed["male"] ==0) & (df_observed["BMI"] > tau)]["BMI"])
m = len(df_observed[df_observed["male"] == 1])
n = len(df_observed[df_observed["male"] == 0])
for _ in range(B):
    p1, p2 = simulate_p_beta(x_1, x_2, m, n)
    phi_flat = logit(p1) - logit(p2)
    phi_list.append(phi_flat)
phi_e = sum(phi_list)/len(phi_list)
a = np.quantile(phi_list, q=(alpha/2))
b = np.quantile(phi_list, q=(1-(alpha/2)))
print(f"The estimator of OR is: {np.around(phi_e, 2)}")
print(f"The credible interval is: [{np.around(a, 2), np.around(b, 2)}]")

The estimator of OR is: 0.55
The credible interval is: [(-0.23, 1.34)]


As we excpect, the odds that men BMI will be above tau is bigger than the odds that women will.<br>
With that being said, we can see that the estimator is much closer to zero and the credible interval is not very informative since it is big and contains zero. This could indicate that the prior we chose might be wrong or that there is no evidence for stochastical differences between the distributions.

## Question 2 c

Now we will assume:<br>
$Z_1^1,...,Z_m^1 \sim Bernouli(p_1)$<br>
$Z_1^2,...,Z_n^2 \sim Bernouli(p_2)$ where:<br>
m is number of males,<br>
n is number of females.<br>
We saw in the lecture that $\pi(p_j)=\sqrt{\frac{1}{p_j(1-p)j)}}$ for j=1,2. <br>
Using these priors we get that the aposteriors distributions of $p_1 \ and \ p_2$ are:<br>
$p1|X^1\sim{Beta(X^{1}_{\tau} + 0.5, m - X^{1}_{\tau} +0.5)}$ <br>
$p2|X^2\sim{Beta(X^{2}_{\tau} + 0.5, n - X^{2}_{\tau} +0.5)}$

In [11]:
B = 10000
phi_jefferys_list = []
for _ in range(B):
    p1, p2 = simulate_p_beta(x_1, x_2, m, n, True)
    phi_jefferys = logit(p1) - logit(p2)
    phi_jefferys_list.append(phi_jefferys)
phi_jefferys_e = sum(phi_jefferys_list)/len(phi_jefferys_list)
a = np.quantile(phi_jefferys_list, q=(alpha/2))
b = np.quantile(phi_jefferys_list, q=(1-(alpha/2)))
print(f"The estimator of OR is: {np.around(phi_jefferys_e, 2)}")
print(f"The credible interval is: [{np.around(a, 2), np.around(b, 2)}]")

The estimator of OR is: 0.56
The credible interval is: [(-0.21, 1.34)]


## Question 2 d

Assuming that now both $p_1 \ and \ p_2$ have priors in the form of Beta distribution and also assuming $Z_i^j \sim Bernouli(p_j)$, we get that the aposterior distributions are: <br>
$p1|X^1, X^1_{old} \sim{Beta(X^{1}_{\tau} + \alpha_1 + 1, m - X^{1}_{\tau} + \beta_1 + 1)}$ <br>
$p2|X^2, X^2_{old} \sim{Beta(X^{2}_{\tau} + \alpha_2 + 1, n - X^{2}_{\tau} + \beta_2 + 1)}$

In [22]:
male_z = df_old_observation[df_old_observation['male'] == 1][['BMI']]
female_z = df_old_observation[df_old_observation['male'] == 0][['BMI']]

In [34]:
male_z['Z'] = male_z[['BMI']].apply(lambda x: x>tau)
female_z['Z'] = female_z[['BMI']].apply(lambda x: x>tau)

In [40]:
male_alpha, male_beta = beta.fit(male_z['Z'])[:2]
female_alpha, female_beta = beta.fit(female_z['Z'])[:2]

In [47]:
print(f"The parameteres for p1 are: alpha={np.around(male_alpha, 2)}, beta={np.around(male_beta, 2)}.\nThe parameteres for p2 are: alpha={np.around(female_alpha, 2)}, beta={np.around(female_beta, 2)}.")

The parameteres for p1 are: alpha=1.48, beta=0.62.
The parameteres for p2 are: alpha=1.46, beta=0.92.


In [48]:
B = 10000
phi_old_data_list = []
for _ in range(B):
    p1, p2 = beta.rvs(male_alpha, male_beta), beta.rvs(female_alpha, female_beta)
    phi_old_data = logit(p1) - logit(p2)
    phi_old_data_list.append(phi_old_data)
phi_old_data_e = sum(phi_old_data_list)/len(phi_old_data_list)
a = np.quantile(phi_old_data_list, q=(alpha/2))
b = np.quantile(phi_old_data_list, q=(1-(alpha/2)))
print(f"The estimator of OR is: {np.around(phi_old_data_e, 2)}")
print(f"The credible interval is: [{np.around(a, 2), np.around(b, 2)}]")

The estimator of OR is: 0.81
The credible interval is: [(-4.14, 6.69)]


## Question 2 d

In [58]:
print(f"Comparing all estimators for phi:\nphi based on MLE={np.around(phi, 2)},\nphi based on flat priors={np.around(phi_e, 2)},\nphi based on Jefferys' prior={np.around(phi_jefferys_e, 2)},\nphi based on old data and beta prior={np.around(phi_old_data_e, 2)}.")

Comparing all estimators for phi:
phi based on MLE=6.56,
phi based on flat priors=0.55,
phi based on Jefferys' prior=0.56,
phi based on old data and beta prior=0.81.


As we can see, MLE got us the highest phi estimation by far. Moreover, we can see that all estimators agree that the odds of males BMI being larger than $\tau$ are greater than the odds of females BMI to pass it. <br>
However, as we saw in previous sections, the credible intervals for all methods other than MLE are quite large and include zero so it might be not informative enough. <br>
Since MLE doesn't assume any prior over $p_1 \ and \ p_2$ it might be better to go with it and say confidently that the odds are in favour of males BMI. <br>
In addition, comparing between flat, Jefferys' and Beta priors over $p_1 \ and \ p_2$ we can see that the flat and Jefferys' priors achieved quite the same estimators while Beta prior achieved an estimator closer to 1. <br>
This could be a result of relying on old data and some knowledge over the parameters which is more informative yet biased than the priors that try to be non-informative.